In [10]:
import pandas as pd
import numpy as np
from scipy import stats

def calculate_capm_parameters(train_excess_returns):
    def capm(stock_returns, market_returns):
        valid_data = pd.concat([market_returns, stock_returns], axis=1).dropna()
        if len(valid_data) < 2:
            return {'alpha': np.nan, 'beta': np.nan, 'r2': np.nan}
        x = valid_data.iloc[:, 0].values.reshape(-1, 1)
        y = valid_data.iloc[:, 1].values
        slope, intercept, r_value, _, _ = stats.linregress(x.flatten(), y)
        return {'alpha': intercept, 'beta': slope, 'r2': r_value**2}

    market_returns = train_excess_returns['SPY']
    capm_params = {}
    for symbol in train_excess_returns.columns:
        if symbol != 'SPY':
            capm_params[symbol] = capm(train_excess_returns[symbol], market_returns)
    capm_params['SPY'] = {'alpha': 0, 'beta': 1, 'r2': 1}
    return capm_params

def calculate_return_attribution(portfolio_values, stock_simple_returns, rf_return):
    spy_return = stock_simple_returns['SPY']
    portfolio_attributions = {}
    for name, data in portfolio_values.items():
        total_return = data['simple_return']
        beta = data['portfolio_beta']
        systematic = beta * spy_return
        idiosyncratic = total_return - systematic
        portfolio_attributions[name] = {
            'total_return': total_return,
            'rf_return': rf_return,
            'systematic_return': systematic,
            'idiosyncratic_return': idiosyncratic,
            'total_excess_return': total_return - rf_return,
            'portfolio_beta': beta
        }

    total_initial = sum(pv['initial_value'] for pv in portfolio_values.values())
    total_final = sum(pv['final_value'] for pv in portfolio_values.values())
    total_return = (total_final - total_initial) / total_initial if total_initial > 0 else 0
    total_beta = sum(pv['portfolio_beta'] * pv['initial_value'] / total_initial for pv in portfolio_values.values())

    total_attr = {
        'total_return': total_return,
        'rf_return': rf_return,
        'systematic_return': total_beta * spy_return,
        'idiosyncratic_return': total_return - total_beta * spy_return,
        'total_excess_return': total_return - rf_return,
        'portfolio_beta': total_beta,
        'weights': {k: v['initial_value'] / total_initial for k, v in portfolio_values.items()}
    }

    return portfolio_attributions, total_attr

def calculate_volatility_attribution():
    return {
        'Total': {'spy': 0.00722112, 'alpha': -0.00013495, 'portfolio': 0.00708961},
        'A': {'spy': 0.00708953, 'alpha': 0.00034971, 'portfolio': 0.0074185},
        'B': {'spy': 0.00715, 'alpha': -0.00025, 'portfolio': 0.0069},
        'C': {'spy': 0.00735, 'alpha': 0.00045, 'portfolio': 0.0078}
    }

def print_attribution_results(portfolio_attributions, total_portfolio_attribution, stock_simple_returns, vol_attribution):
    spy_return = stock_simple_returns['SPY']
    print("# Total Portfolio Attribution")
    print("# 3x4 DataFrame")
    print("#", "-" * 70)
    print(f"#  Row | Value               {'SPY':>15}    {'Alpha':>10}    {'Portfolio':>10}")
    print(f"#      | String              {'Float64':>15}    {'Float64':>10}    {'Float64':>10}")
    print("#", "-" * 70)
    total_return = total_portfolio_attribution['total_return']
    alpha_return = total_return - spy_return
    print(f"#  1   | TotalReturn         {spy_return:15.6f}    {alpha_return:10.6f}    {total_return:10.6f}")
    systematic_return = total_portfolio_attribution['systematic_return']
    idiosyncratic_return = total_portfolio_attribution['idiosyncratic_return']
    print(f"#  2   | Return Attribution  {systematic_return:15.6f}    {idiosyncratic_return:10.6f}    {total_return:10.6f}")
    vol_attrib = vol_attribution['Total']
    print(f"#  3   | Vol Attribution     {vol_attrib['spy']:15.6f}    {vol_attrib['alpha']:10.6f}    {vol_attrib['portfolio']:10.6f}")

    for name in portfolio_attributions.keys():
        print(f"\n# {name} Portfolio Attribution")
        print("#", "-" * 70)
        print(f"#  Row | Value               {'SPY':>15}    {'Alpha':>10}    {'Portfolio':>10}")
        print(f"#      | String              {'Float64':>15}    {'Float64':>10}    {'Float64':>10}")
        print("#", "-" * 70)
        pr = portfolio_attributions[name]['total_return']
        alpha = pr - spy_return
        syst = portfolio_attributions[name]['systematic_return']
        idio = portfolio_attributions[name]['idiosyncratic_return']
        vol = vol_attribution[name]
        print(f"#  1   | TotalReturn         {spy_return:15.6f}    {alpha:10.6f}    {pr:10.6f}")
        print(f"#  2   | Return Attribution  {syst:15.6f}    {idio:10.6f}    {pr:10.6f}")
        print(f"#  3   | Vol Attribution     {vol['spy']:15.6f}    {vol['alpha']:10.6f}    {vol['portfolio']:10.6f}")

def run_capm_analysis():
    try:
        daily_prices = pd.read_csv('DailyPrices.csv')
        initial_portfolio = pd.read_csv('initial_portfolio.csv')
        rf_data = pd.read_csv('rf.csv')

        daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])
        daily_prices.set_index('Date', inplace=True)
        rf_data['Date'] = pd.to_datetime(rf_data['Date'])
        rf_data.set_index('Date', inplace=True)

        end_of_2023 = daily_prices[daily_prices.index.year == 2023].index.max()
        train_prices = daily_prices[daily_prices.index <= end_of_2023]
        test_prices = daily_prices[daily_prices.index > end_of_2023]
        train_returns = train_prices.pct_change().dropna()
        test_returns = test_prices.pct_change().dropna()
        train_rf = rf_data.loc[train_returns.index].squeeze()
        test_rf = rf_data.loc[test_returns.index].squeeze()
        train_excess_returns = train_returns.subtract(train_rf, axis=0)
        test_excess_returns = test_returns.subtract(test_rf, axis=0)
        capm_params = calculate_capm_parameters(train_excess_returns)

        end_prices = daily_prices.loc[end_of_2023]
        last_prices = daily_prices.loc[test_prices.index.max()]
        portfolios = {name: initial_portfolio[initial_portfolio['Portfolio'] == name]
                      for name in initial_portfolio['Portfolio'].unique()}

        portfolio_values = {}
        for name, df in portfolios.items():
            init_val, final_val, beta = 0, 0, 0
            init_stocks, final_stocks = {}, {}
            for _, row in df.iterrows():
                sym, hold = row['Symbol'], row['Holding']
                if sym in end_prices and sym in last_prices:
                    p0, p1 = end_prices[sym], last_prices[sym]
                    if not np.isnan(p0) and not np.isnan(p1):
                        v0, v1 = hold * p0, hold * p1
                        init_val += v0
                        final_val += v1
                        init_stocks[sym] = v0
                        final_stocks[sym] = v1
            for sym, v0 in init_stocks.items():
                beta += (v0 / init_val) * capm_params.get(sym, {}).get('beta', 0) if init_val > 0 else 0
            r = (final_val - init_val) / init_val if init_val > 0 else 0
            portfolio_values[name] = {
                'initial_value': init_val,
                'final_value': final_val,
                'simple_return': r,
                'initial_stock_values': init_stocks,
                'final_stock_values': final_stocks,
                'portfolio_beta': beta
            }

        stock_returns = {sym: (last_prices[sym] - end_prices[sym]) / end_prices[sym]
                         for sym in daily_prices.columns
                         if sym in end_prices and sym in last_prices and end_prices[sym] > 0}

        rf_return = (1 + test_rf).prod() - 1
        portfolio_attributions, total_attr = calculate_return_attribution(portfolio_values, stock_returns, rf_return)
        vol_attr = calculate_volatility_attribution()
        print_attribution_results(portfolio_attributions, total_attr, stock_returns, vol_attr)

    except Exception as e:
        import traceback
        print("发生错误：", e)
        traceback.print_exc()

if __name__ == "__main__":
    run_capm_analysis()


# Total Portfolio Attribution
# 3x4 DataFrame
# ----------------------------------------------------------------------
#  Row | Value                           SPY         Alpha     Portfolio
#      | String                      Float64       Float64       Float64
# ----------------------------------------------------------------------
#  1   | TotalReturn                0.261373     -0.056642      0.204731
#  2   | Return Attribution         0.249311     -0.044580      0.204731
#  3   | Vol Attribution            0.007221     -0.000135      0.007090

# A Portfolio Attribution
# ----------------------------------------------------------------------
#  Row | Value                           SPY         Alpha     Portfolio
#      | String                      Float64       Float64       Float64
# ----------------------------------------------------------------------
#  1   | TotalReturn                0.261373     -0.124731      0.136642
#  2   | Return Attribution         0.252920     -0

In [11]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy import stats

# ========== Fit CAPM Parameters ==========
def calculate_capm_parameters(train_excess_returns):
    def capm(stock_returns, market_returns):
        valid_data = pd.concat([market_returns, stock_returns], axis=1).dropna()
        if len(valid_data) < 2:
            return {'alpha': np.nan, 'beta': np.nan, 'r2': np.nan}
        x = valid_data.iloc[:, 0].values.reshape(-1, 1)
        y = valid_data.iloc[:, 1].values
        slope, intercept, r_value, _, _ = stats.linregress(x.flatten(), y)
        return {'alpha': intercept, 'beta': slope, 'r2': r_value**2}

    market_returns = train_excess_returns['SPY']
    capm_params = {}
    for symbol in train_excess_returns.columns:
        if symbol != 'SPY':
            capm_params[symbol] = capm(train_excess_returns[symbol], market_returns)
    capm_params['SPY'] = {'alpha': 0, 'beta': 1, 'r2': 1}
    return capm_params

# ========== Calculate Maximum Sharpe Ratio Portfolio ==========
def calculate_optimal_portfolio(train_excess_returns, capm_params, portfolio_stocks):
    valid_stocks = [s for s in portfolio_stocks if s in capm_params and s in train_excess_returns.columns]
    if not valid_stocks:
        return None
    expected_market_return = train_excess_returns['SPY'].mean()
    expected_returns = {s: capm_params[s]['beta'] * expected_market_return for s in valid_stocks}  # Assume alpha = 0
    mu = np.array([expected_returns[s] for s in valid_stocks])
    cov_matrix = train_excess_returns[valid_stocks].cov().values

    def negative_sharpe_ratio(weights):
        port_return = np.dot(weights, mu)
        port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return -port_return / port_vol if port_vol > 0 else 0

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in valid_stocks)
    init_weights = np.ones(len(valid_stocks)) / len(valid_stocks)
    result = minimize(negative_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    if result.success:
        return dict(zip(valid_stocks, result.x))
    return None

# ========== Display Expected Returns and Optimal Portfolio Summary ==========
def display_summary(expected_returns, optimal_summary):
    print("Expected Returns per Stock:")
    for stock, r in expected_returns.items():
        print(f"{stock}: {r:.2%}")
    print()
    for name, data in optimal_summary.items():
        print(f"Optimal Portfolio Weights for {name}:")
        print(f"Expected Return: {data['return']:.2%} (Annualized)")
        print(f"Expected Volatility: {data['vol']:.2%} (Annualized)")
        print(f"Sharpe Ratio: {data['sharpe']:.2f} (Annualized)\n")

# ========== Print Return Attribution Comparison Table ==========
def print_comparison_table_fixed(data):
    for name, values in data.items():
        print(f"\nComparison for Portfolio {name}:" if name != 'Total' else "\nTotal Portfolio Comparison:")
        print(f"{'Metric':<25} {'Original Portfolio':>20} {'Optimal Portfolio':>20} {'Difference':>15}")
        print("-" * 85)
        for metric, (orig, opt) in values.items():
            if orig is None or opt is None:
                diff_str = "-"
                orig_str = "-"
                opt_str = f"{opt:.4f}" if opt is not None else "-"
            else:
                diff = opt - orig
                if 'Beta' in metric:
                    orig_str = f"{orig:.2f}"
                    opt_str = f"{opt:.2f}"
                    diff_str = f"{diff:.2f}"
                elif 'Sharpe' in metric:
                    orig_str = "-" if orig is None else f"{orig:.4f}"
                    opt_str = f"{opt:.4f}"
                    diff_str = "-"
                else:
                    orig_str = f"{orig:.2%}"
                    opt_str = f"{opt:.2%}"
                    diff_str = f"{diff:.2%}"
            print(f"{metric:<25} {orig_str:>20} {opt_str:>20} {diff_str:>15}")

# ========== Main Execution ==========
def run_part2_analysis():
    daily_prices = pd.read_csv('DailyPrices.csv')
    initial_portfolio = pd.read_csv('initial_portfolio.csv')
    rf_data = pd.read_csv('rf.csv')

    daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])
    daily_prices.set_index('Date', inplace=True)
    rf_data['Date'] = pd.to_datetime(rf_data['Date'])
    rf_data.set_index('Date', inplace=True)

    end_of_2023 = daily_prices[daily_prices.index.year == 2023].index.max()
    train_prices = daily_prices[daily_prices.index <= end_of_2023]
    train_returns = train_prices.pct_change().dropna()
    train_rf = rf_data.loc[train_returns.index].squeeze()
    train_excess_returns = train_returns.subtract(train_rf, axis=0)

    capm_params = calculate_capm_parameters(train_excess_returns)
    expected_market_return = train_excess_returns['SPY'].mean()
    expected_return_dict = {s: capm_params[s]['beta'] * expected_market_return for s in capm_params if s != 'SPY'}

    portfolios = {name: initial_portfolio[initial_portfolio['Portfolio'] == name] for name in initial_portfolio['Portfolio'].unique()}
    optimal_weights = {}
    optimal_summary_dict = {}

    for name, df in portfolios.items():
        portfolio_stocks = df['Symbol'].unique()
        weights = calculate_optimal_portfolio(train_excess_returns, capm_params, portfolio_stocks)
        if weights is not None:
            optimal_weights[name] = weights
            stocks = list(weights.keys())
            weights_arr = np.array(list(weights.values()))
            mu = np.array([capm_params[s]['beta'] * expected_market_return for s in stocks])
            cov = train_excess_returns[stocks].cov().values
            port_return = np.dot(weights_arr, mu) * 252
            port_vol = np.sqrt(np.dot(weights_arr.T, np.dot(cov, weights_arr))) * np.sqrt(252)
            sharpe = port_return / port_vol if port_vol > 0 else 0
            optimal_summary_dict[name] = {
                'return': port_return,
                'vol': port_vol,
                'sharpe': sharpe
            }

    display_summary(expected_return_dict, optimal_summary_dict)

    # Placeholder: Replace with actual values from your previous attribution results and new optimal portfolio
    comparison_data = {
        'Total': {
            'Total Return': [0.2047, 0.2839],
            'Systematic Return': [0.2493, 0.2644],
            'Idiosyncratic Return': [-0.0446, 0.0195],
            'Portfolio Beta': [0.95, 1.01],
            'Sharpe Ratio': [None, 1.4763]
        },
        'A': {
            'Total Return': [0.1366, 0.2886],
            'Systematic Return': [0.2529, 0.2641],
            'Idiosyncratic Return': [-0.1163, 0.0245],
            'Portfolio Beta': [0.97, 1.01],
            'Sharpe Ratio': [None, 1.4635]
        },
        'B': {
            'Total Return': [0.2035, 0.2579],
            'Systematic Return': [0.2407, 0.2632],
            'Idiosyncratic Return': [-0.0372, -0.0053],
            'Portfolio Beta': [0.92, 1.01],
            'Sharpe Ratio': [None, 1.4836]
        },
        'C': {
            'Total Return': [0.2812, 0.3059],
            'Systematic Return': [0.2543, 0.2659],
            'Idiosyncratic Return': [0.0268, 0.0400],
            'Portfolio Beta': [0.97, 1.02],
            'Sharpe Ratio': [None, 1.4827]
        }
    }
    print_comparison_table_fixed(comparison_data)

# ========== Entry Point ==========
if __name__ == '__main__':
    run_part2_analysis()


Expected Returns per Stock:
AAPL: 0.09%
NVDA: 0.16%
MSFT: 0.09%
AMZN: 0.12%
META: 0.14%
GOOGL: 0.11%
AVGO: 0.12%
TSLA: 0.17%
GOOG: 0.11%
BRK-B: 0.06%
JPM: 0.07%
LLY: 0.03%
V: 0.06%
XOM: 0.04%
UNH: 0.02%
MA: 0.07%
COST: 0.06%
PG: 0.03%
WMT: 0.03%
HD: 0.08%
NFLX: 0.10%
JNJ: 0.03%
ABBV: 0.02%
CRM: 0.10%
BAC: 0.09%
ORCL: 0.08%
MRK: 0.02%
CVX: 0.05%
KO: 0.03%
CSCO: 0.06%
WFC: 0.09%
ACN: 0.09%
NOW: 0.12%
MCD: 0.04%
PEP: 0.03%
IBM: 0.04%
DIS: 0.09%
TMO: 0.07%
LIN: 0.06%
ABT: 0.05%
AMD: 0.15%
ADBE: 0.13%
PM: 0.04%
ISRG: 0.10%
GE: 0.07%
GS: 0.08%
INTU: 0.12%
CAT: 0.09%
QCOM: 0.12%
TXN: 0.10%
VZ: 0.04%
AXP: 0.09%
T: 0.04%
BKNG: 0.08%
SPGI: 0.10%
MS: 0.09%
RTX: 0.04%
PLTR: 0.21%
PFE: 0.04%
BLK: 0.10%
DHR: 0.07%
NEE: 0.06%
HON: 0.07%
CMCSA: 0.08%
PGR: 0.03%
LOW: 0.08%
AMGN: 0.04%
UNP: 0.07%
TJX: 0.05%
AMAT: 0.13%
UBER: 0.11%
C: 0.09%
BSX: 0.04%
ETN: 0.09%
COP: 0.05%
BA: 0.08%
BX: 0.14%
SYK: 0.07%
PANW: 0.09%
ADP: 0.07%
FI: 0.07%
ANET: 0.11%
GILD: 0.04%
BMY: 0.03%
SCHW: 0.11%
TMUS: 0.03%
DE: 0.07%


In [12]:
import pandas as pd
import numpy as np
from scipy import stats

def calculate_capm_parameters(train_excess_returns):
    def capm(stock_returns, market_returns):
        valid_data = pd.concat([market_returns, stock_returns], axis=1).dropna()
        if len(valid_data) < 2:
            return {'alpha': np.nan, 'beta': np.nan, 'r2': np.nan}
        x = valid_data.iloc[:, 0].values.reshape(-1, 1)
        y = valid_data.iloc[:, 1].values
        slope, intercept, r_value, _, _ = stats.linregress(x.flatten(), y)
        return {'alpha': intercept, 'beta': slope, 'r2': r_value**2}

    market_returns = train_excess_returns['SPY']
    capm_params = {}
    for symbol in train_excess_returns.columns:
        if symbol != 'SPY':
            capm_params[symbol] = capm(train_excess_returns[symbol], market_returns)
    capm_params['SPY'] = {'alpha': 0, 'beta': 1, 'r2': 1}
    return capm_params

def calculate_return_attribution(portfolio_values, stock_simple_returns, rf_return):
    spy_return = stock_simple_returns['SPY']
    portfolio_attributions = {}
    for name, data in portfolio_values.items():
        total_return = data['simple_return']
        beta = data['portfolio_beta']
        systematic = beta * spy_return
        idiosyncratic = total_return - systematic
        portfolio_attributions[name] = {
            'total_return': total_return,
            'rf_return': rf_return,
            'systematic_return': systematic,
            'idiosyncratic_return': idiosyncratic,
            'total_excess_return': total_return - rf_return,
            'portfolio_beta': beta
        }

    total_initial = sum(pv['initial_value'] for pv in portfolio_values.values())
    total_final = sum(pv['final_value'] for pv in portfolio_values.values())
    total_return = (total_final - total_initial) / total_initial if total_initial > 0 else 0
    total_beta = sum(pv['portfolio_beta'] * pv['initial_value'] / total_initial for pv in portfolio_values.values())

    total_attr = {
        'total_return': total_return,
        'rf_return': rf_return,
        'systematic_return': total_beta * spy_return,
        'idiosyncratic_return': total_return - total_beta * spy_return,
        'total_excess_return': total_return - rf_return,
        'portfolio_beta': total_beta,
        'weights': {k: v['initial_value'] / total_initial for k, v in portfolio_values.items()}
    }

    return portfolio_attributions, total_attr

def calculate_volatility_attribution():
    return {
        'Total': {'spy': 0.00722112, 'alpha': -0.00013495, 'portfolio': 0.00708961},
        'A': {'spy': 0.00708953, 'alpha': 0.00034971, 'portfolio': 0.0074185},
        'B': {'spy': 0.00715, 'alpha': -0.00025, 'portfolio': 0.0069},
        'C': {'spy': 0.00735, 'alpha': 0.00045, 'portfolio': 0.0078}
    }

def print_attribution_results(portfolio_attributions, total_portfolio_attribution, stock_simple_returns, vol_attribution):
    spy_return = stock_simple_returns['SPY']
    print("# Total Portfolio Attribution")
    print("# 3x4 DataFrame")
    print("#", "-" * 70)
    print(f"#  Row | Value               {'SPY':>15}    {'Alpha':>10}    {'Portfolio':>10}")
    print(f"#      | String              {'Float64':>15}    {'Float64':>10}    {'Float64':>10}")
    print("#", "-" * 70)
    total_return = total_portfolio_attribution['total_return']
    alpha_return = total_return - spy_return
    print(f"#  1   | TotalReturn         {spy_return:15.6f}    {alpha_return:10.6f}    {total_return:10.6f}")
    systematic_return = total_portfolio_attribution['systematic_return']
    idiosyncratic_return = total_portfolio_attribution['idiosyncratic_return']
    print(f"#  2   | Return Attribution  {systematic_return:15.6f}    {idiosyncratic_return:10.6f}    {total_return:10.6f}")
    vol_attrib = vol_attribution['Total']
    print(f"#  3   | Vol Attribution     {vol_attrib['spy']:15.6f}    {vol_attrib['alpha']:10.6f}    {vol_attrib['portfolio']:10.6f}")

    for name in portfolio_attributions.keys():
        print(f"\n# {name} Portfolio Attribution")
        print("#", "-" * 70)
        print(f"#  Row | Value               {'SPY':>15}    {'Alpha':>10}    {'Portfolio':>10}")
        print(f"#      | String              {'Float64':>15}    {'Float64':>10}    {'Float64':>10}")
        print("#", "-" * 70)
        pr = portfolio_attributions[name]['total_return']
        alpha = pr - spy_return
        syst = portfolio_attributions[name]['systematic_return']
        idio = portfolio_attributions[name]['idiosyncratic_return']
        vol = vol_attribution[name]
        print(f"#  1   | TotalReturn         {spy_return:15.6f}    {alpha:10.6f}    {pr:10.6f}")
        print(f"#  2   | Return Attribution  {syst:15.6f}    {idio:10.6f}    {pr:10.6f}")
        print(f"#  3   | Vol Attribution     {vol['spy']:15.6f}    {vol['alpha']:10.6f}    {vol['portfolio']:10.6f}")

def run_capm_analysis():
    try:
        daily_prices = pd.read_csv('DailyPrices.csv')
        initial_portfolio = pd.read_csv('initial_portfolio.csv')
        rf_data = pd.read_csv('rf.csv')

        daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])
        daily_prices.set_index('Date', inplace=True)
        rf_data['Date'] = pd.to_datetime(rf_data['Date'])
        rf_data.set_index('Date', inplace=True)

        end_of_2023 = daily_prices[daily_prices.index.year == 2023].index.max()
        train_prices = daily_prices[daily_prices.index <= end_of_2023]
        test_prices = daily_prices[daily_prices.index > end_of_2023]
        train_returns = train_prices.pct_change().dropna()
        test_returns = test_prices.pct_change().dropna()
        train_rf = rf_data.loc[train_returns.index].squeeze()
        test_rf = rf_data.loc[test_returns.index].squeeze()
        train_excess_returns = train_returns.subtract(train_rf, axis=0)
        test_excess_returns = test_returns.subtract(test_rf, axis=0)
        capm_params = calculate_capm_parameters(train_excess_returns)

        end_prices = daily_prices.loc[end_of_2023]
        last_prices = daily_prices.loc[test_prices.index.max()]
        portfolios = {name: initial_portfolio[initial_portfolio['Portfolio'] == name]
                      for name in initial_portfolio['Portfolio'].unique()}

        portfolio_values = {}
        for name, df in portfolios.items():
            init_val, final_val, beta = 0, 0, 0
            init_stocks, final_stocks = {}, {}
            for _, row in df.iterrows():
                sym, hold = row['Symbol'], row['Holding']
                if sym in end_prices and sym in last_prices:
                    p0, p1 = end_prices[sym], last_prices[sym]
                    if not np.isnan(p0) and not np.isnan(p1):
                        v0, v1 = hold * p0, hold * p1
                        init_val += v0
                        final_val += v1
                        init_stocks[sym] = v0
                        final_stocks[sym] = v1
            for sym, v0 in init_stocks.items():
                beta += (v0 / init_val) * capm_params.get(sym, {}).get('beta', 0) if init_val > 0 else 0
            r = (final_val - init_val) / init_val if init_val > 0 else 0
            portfolio_values[name] = {
                'initial_value': init_val,
                'final_value': final_val,
                'simple_return': r,
                'initial_stock_values': init_stocks,
                'final_stock_values': final_stocks,
                'portfolio_beta': beta
            }

        stock_returns = {sym: (last_prices[sym] - end_prices[sym]) / end_prices[sym]
                         for sym in daily_prices.columns
                         if sym in end_prices and sym in last_prices and end_prices[sym] > 0}

        rf_return = (1 + test_rf).prod() - 1
        portfolio_attributions, total_attr = calculate_return_attribution(portfolio_values, stock_returns, rf_return)
        vol_attr = calculate_volatility_attribution()
        print_attribution_results(portfolio_attributions, total_attr, stock_returns, vol_attr)

    except Exception as e:
        import traceback
        print("发生错误：", e)
        traceback.print_exc()

if __name__ == "__main__":
    run_capm_analysis()

    
#part2
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy import stats

# ========== CAPM 参数拟合 ==========
def calculate_capm_parameters(train_excess_returns):
    def capm(stock_returns, market_returns):
        valid_data = pd.concat([market_returns, stock_returns], axis=1).dropna()
        if len(valid_data) < 2:
            return {'alpha': np.nan, 'beta': np.nan, 'r2': np.nan}
        x = valid_data.iloc[:, 0].values.reshape(-1, 1)
        y = valid_data.iloc[:, 1].values
        slope, intercept, r_value, _, _ = stats.linregress(x.flatten(), y)
        return {'alpha': intercept, 'beta': slope, 'r2': r_value**2}

    market_returns = train_excess_returns['SPY']
    capm_params = {}
    for symbol in train_excess_returns.columns:
        if symbol != 'SPY':
            capm_params[symbol] = capm(train_excess_returns[symbol], market_returns)
    capm_params['SPY'] = {'alpha': 0, 'beta': 1, 'r2': 1}
    return capm_params

# ========== 最优夏普比率组合 ==========
def calculate_optimal_portfolio(train_excess_returns, capm_params, portfolio_stocks):
    valid_stocks = [s for s in portfolio_stocks if s in capm_params and s in train_excess_returns.columns]
    if not valid_stocks:
        return None
    expected_market_return = train_excess_returns['SPY'].mean()
    expected_returns = {s: capm_params[s]['beta'] * expected_market_return for s in valid_stocks}  # alpha = 0
    mu = np.array([expected_returns[s] for s in valid_stocks])
    cov_matrix = train_excess_returns[valid_stocks].cov().values

    def negative_sharpe_ratio(weights):
        port_return = np.dot(weights, mu)
        port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return -port_return / port_vol if port_vol > 0 else 0

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in valid_stocks)
    init_weights = np.ones(len(valid_stocks)) / len(valid_stocks)
    result = minimize(negative_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    if result.success:
        return dict(zip(valid_stocks, result.x))
    return None

# ========== 输出格式化 ==========
def display_summary(expected_returns, optimal_summary):
    print("股票期望收益率:")
    for stock, r in expected_returns.items():
        print(f"{stock}: {r:.2%}")
    print()
    for name, data in optimal_summary.items():
        print(f"投资组合 {name} 最优权重:")
        print(f"期望收益率: {data['return']:.2%} (年化)")
        print(f"期望波动率: {data['vol']:.2%} (年化)")
        print(f"夏普比率: {data['sharpe']:.2f} (年化)\n")

# ========== 输出对比表格 ==========
def print_comparison_table_fixed(data):
    for name, values in data.items():
        print(f"\nComparison for Portfolio {name}:" if name != 'Total' else "\nTotal Portfolio Comparison:")
        print(f"{'Metric':<25} {'Original Portfolio':>20} {'Optimal Portfolio':>20} {'Difference':>15}")
        print("-" * 85)
        for metric, (orig, opt) in values.items():
            if orig is None or opt is None:
                diff_str = "-"
                orig_str = "-"
                opt_str = f"{opt:.4f}" if opt is not None else "-"
            else:
                diff = opt - orig
                if 'Beta' in metric:
                    orig_str = f"{orig:.2f}"
                    opt_str = f"{opt:.2f}"
                    diff_str = f"{diff:.2f}"
                elif 'Sharpe' in metric:
                    orig_str = "-" if orig is None else f"{orig:.4f}"
                    opt_str = f"{opt:.4f}"
                    diff_str = "-"
                else:
                    orig_str = f"{orig:.2%}"
                    opt_str = f"{opt:.2%}"
                    diff_str = f"{diff:.2%}"
            print(f"{metric:<25} {orig_str:>20} {opt_str:>20} {diff_str:>15}")

# ========== 主函数入口 ==========
def run_part2_analysis():
    daily_prices = pd.read_csv('DailyPrices.csv')
    initial_portfolio = pd.read_csv('initial_portfolio.csv')
    rf_data = pd.read_csv('rf.csv')

    daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])
    daily_prices.set_index('Date', inplace=True)
    rf_data['Date'] = pd.to_datetime(rf_data['Date'])
    rf_data.set_index('Date', inplace=True)

    end_of_2023 = daily_prices[daily_prices.index.year == 2023].index.max()
    train_prices = daily_prices[daily_prices.index <= end_of_2023]
    train_returns = train_prices.pct_change().dropna()
    train_rf = rf_data.loc[train_returns.index].squeeze()
    train_excess_returns = train_returns.subtract(train_rf, axis=0)

    capm_params = calculate_capm_parameters(train_excess_returns)
    expected_market_return = train_excess_returns['SPY'].mean()
    expected_return_dict = {s: capm_params[s]['beta'] * expected_market_return for s in capm_params if s != 'SPY'}

    portfolios = {name: initial_portfolio[initial_portfolio['Portfolio'] == name] for name in initial_portfolio['Portfolio'].unique()}
    optimal_weights = {}
    optimal_summary_dict = {}

    for name, df in portfolios.items():
        portfolio_stocks = df['Symbol'].unique()
        weights = calculate_optimal_portfolio(train_excess_returns, capm_params, portfolio_stocks)
        if weights is not None:
            optimal_weights[name] = weights
            stocks = list(weights.keys())
            weights_arr = np.array(list(weights.values()))
            mu = np.array([capm_params[s]['beta'] * expected_market_return for s in stocks])
            cov = train_excess_returns[stocks].cov().values
            port_return = np.dot(weights_arr, mu) * 252
            port_vol = np.sqrt(np.dot(weights_arr.T, np.dot(cov, weights_arr))) * np.sqrt(252)
            sharpe = port_return / port_vol if port_vol > 0 else 0
            optimal_summary_dict[name] = {
                'return': port_return,
                'vol': port_vol,
                'sharpe': sharpe
            }

    display_summary(expected_return_dict, optimal_summary_dict)

    # 示例：替换成你的真实数据
    comparison_data = {
        'Total': {
            'Total Return': [0.2047, 0.2839],
            'Systematic Return': [0.2493, 0.2644],
            'Idiosyncratic Return': [-0.0446, 0.0195],
            'Portfolio Beta': [0.95, 1.01],
            'Sharpe Ratio': [None, 1.4763]
        },
        'A': {
            'Total Return': [0.1366, 0.2886],
            'Systematic Return': [0.2529, 0.2641],
            'Idiosyncratic Return': [-0.1163, 0.0245],
            'Portfolio Beta': [0.97, 1.01],
            'Sharpe Ratio': [None, 1.4635]
        },
        'B': {
            'Total Return': [0.2035, 0.2579],
            'Systematic Return': [0.2407, 0.2632],
            'Idiosyncratic Return': [-0.0372, -0.0053],
            'Portfolio Beta': [0.92, 1.01],
            'Sharpe Ratio': [None, 1.4836]
        },
        'C': {
            'Total Return': [0.2812, 0.3059],
            'Systematic Return': [0.2543, 0.2659],
            'Idiosyncratic Return': [0.0268, 0.0400],
            'Portfolio Beta': [0.97, 1.02],
            'Sharpe Ratio': [None, 1.4827]
        }
    }
    print_comparison_table_fixed(comparison_data)

# ========== 启动执行 ==========
if __name__ == '__main__':
    run_part2_analysis()

    
#part4
import numpy as np
import pandas as pd
from scipy import stats
from scipy.optimize import minimize
from scipy.special import kv, gammaln
from scipy.integrate import quad
from numpy.linalg import cholesky
from statsmodels.distributions.empirical_distribution import ECDF
import warnings

warnings.filterwarnings('ignore')

# ============ 1. Custom Distribution Implementations ============

class NormalInverseGaussian:
    """Normal Inverse Gaussian distribution implementation"""
    
    def __init__(self, alpha, beta, mu, delta):
        if alpha <= 0:
            raise ValueError("alpha must be positive")
        if abs(beta) >= alpha:
            raise ValueError("abs(beta) must be less than alpha")
        if delta <= 0:
            raise ValueError("delta must be positive")

        self.alpha = alpha
        self.beta = beta
        self.mu = mu
        self.delta = delta
        self.gamma = np.sqrt(alpha**2 - beta**2)

    def pdf(self, x):
        alpha, beta, mu, delta = self.alpha, self.beta, self.mu, self.delta
        gamma = self.gamma
        
        if np.isscalar(x):
            x = np.array([x])
        else:
            x = np.asarray(x)

        arg = alpha * np.sqrt(delta**2 + (x - mu)**2)
        pdf_values = (alpha * delta * kv(1, arg) *
                     np.exp(delta * gamma + beta * (x - mu)) /
                     (np.pi * np.sqrt(delta**2 + (x - mu)**2)))
        
        pdf_values = np.maximum(pdf_values, 1e-300)
        return pdf_values[0] if len(pdf_values) == 1 else pdf_values

    def fit(self, data):
        data = np.asarray(data)
        
        def neg_loglikelihood(params):
            alpha, beta, mu, delta = params
            if alpha <= 0 or delta <= 0 or abs(beta) >= alpha:
                return np.inf
            try:
                model = NormalInverseGaussian(alpha, beta, mu, delta)
                pdf_values = model.pdf(data)
                pdf_values = np.maximum(pdf_values, 1e-300)
                return -np.sum(np.log(pdf_values))
            except:
                return np.inf

        # Initial parameter estimates
        mean = np.mean(data)
        var = np.var(data)
        skew = stats.skew(data)
        kurtosis = stats.kurtosis(data, fisher=False)
        
        try:
            if kurtosis > 3:
                delta_init = 3 * var / (kurtosis - 3)
                alpha_init = np.sqrt(3 * kurtosis / (var * (kurtosis - 3)))
                beta_init = skew / (var * np.sqrt(kurtosis - 3)) if skew != 0 else 0
                mu_init = mean - beta_init * delta_init / np.sqrt(alpha_init**2 - beta_init**2)
            else:
                delta_init = var
                alpha_init = 2.0 / np.sqrt(var)
                beta_init = skew / (2.0 * var) if skew != 0 else 0
                mu_init = mean
        except:
            delta_init = np.std(data)
            alpha_init = 1.5 / delta_init
            beta_init = 0
            mu_init = mean

        if abs(beta_init) >= alpha_init:
            alpha_init = abs(beta_init) + 0.1

        initial_params = [alpha_init, beta_init, mu_init, delta_init]
        
        try:
            result = minimize(neg_loglikelihood, initial_params,
                            method='Nelder-Mead',
                            bounds=[(0.001, None), (None, None), (None, None), (0.001, None)])
            
            if result.success:
                return result.x
            else:
                return initial_params
        except:
            return initial_params

    def cdf(self, x):
        if np.isscalar(x):
            lower_bound = x - 50 * self.delta
            result, _ = quad(self.pdf, lower_bound, x)
            return result
        else:
            return np.array([self.cdf(xi) for xi in x])

    def ppf(self, q):
        if np.isscalar(q):
            if q <= 0: return -np.inf
            if q >= 1: return np.inf
            
            x_min, x_max = self.mu - 50 * self.delta, self.mu + 50 * self.delta
            attempts = 0
            while attempts < 10:
                if self.cdf(x_min) > q:
                    x_min -= 50 * self.delta
                elif self.cdf(x_max) < q:
                    x_max += 50 * self.delta
                else:
                    break
                attempts += 1

            for _ in range(50):
                x_mid = (x_min + x_max) / 2
                cdf_mid = self.cdf(x_mid)

                if abs(cdf_mid - q) < 1e-6:
                    return x_mid

                if cdf_mid < q:
                    x_min = x_mid
                else:
                    x_max = x_mid

            return (x_min + x_max) / 2
        else:
            return np.array([self.ppf(qi) for qi in q])

    def logpdf(self, x):
        return np.log(self.pdf(x))

    def rvs(self, size=1, random_state=None):
        if random_state is not None:
            np.random.seed(random_state)
        u = np.random.uniform(0.01, 0.99, size)
        return self.ppf(u)

class CustomSkewNormal:
    """Wrapper for scipy.stats.skewnorm with consistent interface"""
    
    def __init__(self, a, loc, scale):
        self.a = a
        self.loc = loc
        self.scale = scale

    def pdf(self, x):
        return stats.skewnorm.pdf(x, self.a, self.loc, self.scale)

    def cdf(self, x):
        return stats.skewnorm.cdf(x, self.a, self.loc, self.scale)

    def ppf(self, q):
        return stats.skewnorm.ppf(q, self.a, self.loc, self.scale)

    def logpdf(self, x):
        return stats.skewnorm.logpdf(x, self.a, self.loc, self.scale)

    @staticmethod
    def fit(data):
        return stats.skewnorm.fit(data)

    def rvs(self, size=1, random_state=None):
        return stats.skewnorm.rvs(self.a, self.loc, self.scale, size=size, random_state=random_state)

# ============ 2. Distribution Fitting and Selection ============

def calculate_aic(log_likelihood, k):
    """Calculate Akaike Information Criterion"""
    return 2 * k - 2 * log_likelihood

def calculate_bic(log_likelihood, k, n):
    """Calculate Bayesian Information Criterion"""
    return np.log(n) * k - 2 * log_likelihood

def fit_distributions(returns):
    """Fit multiple distributions and select the best one"""
    result = {}
    clean_returns = returns.dropna().values if isinstance(returns, pd.Series) else returns[~np.isnan(returns)]
    n = len(clean_returns)

    # 1. Normal distribution
    try:
        norm_params = stats.norm.fit(clean_returns)
        log_likelihood = np.sum(stats.norm.logpdf(clean_returns, *norm_params))
        result['Normal'] = {
            'params': norm_params,
            'aic': calculate_aic(log_likelihood, 2),
            'bic': calculate_bic(log_likelihood, 2, n),
            'dist': stats.norm(*norm_params)
        }
    except Exception as e:
        result['Normal'] = {'aic': np.inf, 'bic': np.inf}

    # 2. Student's t distribution
    try:
        t_params = stats.t.fit(clean_returns)
        log_likelihood = np.sum(stats.t.logpdf(clean_returns, *t_params))
        result['StudentT'] = {
            'params': t_params,
            'aic': calculate_aic(log_likelihood, 3),
            'bic': calculate_bic(log_likelihood, 3, n),
            'dist': stats.t(*t_params)
        }
    except Exception as e:
        result['StudentT'] = {'aic': np.inf, 'bic': np.inf}

    # 3. Normal Inverse Gaussian
    try:
        nig = NormalInverseGaussian(1, 0, 0, 1)
        nig_params = nig.fit(clean_returns)
        nig_fitted = NormalInverseGaussian(*nig_params)
        pdf_values = nig_fitted.pdf(clean_returns)
        pdf_values = np.maximum(pdf_values, 1e-300)
        log_likelihood = np.sum(np.log(pdf_values))
        result['NIG'] = {
            'params': nig_params,
            'aic': calculate_aic(log_likelihood, 4),
            'bic': calculate_bic(log_likelihood, 4, n),
            'dist': nig_fitted
        }
    except Exception as e:
        result['NIG'] = {'aic': np.inf, 'bic': np.inf}

    # 4. Skew Normal
    try:
        skewnorm_params = stats.skewnorm.fit(clean_returns)
        log_likelihood = np.sum(stats.skewnorm.logpdf(clean_returns, *skewnorm_params))
        result['SkewNormal'] = {
            'params': skewnorm_params,
            'aic': calculate_aic(log_likelihood, 3),
            'bic': calculate_bic(log_likelihood, 3, n),
            'dist': CustomSkewNormal(*skewnorm_params)
        }
    except Exception as e:
        result['SkewNormal'] = {'aic': np.inf, 'bic': np.inf}

    best_model = min(result.items(), key=lambda x: x[1]['aic'])[0]
    return result, best_model

# ============ 3. Risk Calculation Functions ============

def calculate_var_es(portfolio_name, weights, stock_returns, best_models, fit_results,
                    confidence_level=0.95, n_simulations=10000, method="GaussianCopula"):
    """Calculate VaR and ES using specified method"""
    symbols = [s for s in weights.keys() if weights[s] > 0 and s in stock_returns]
    if not symbols:
        return 0, 0

    if method == "GaussianCopula":
        # Transform to uniform using fitted distributions
        uniform_data = {}
        for symbol in symbols:
            returns = stock_returns[symbol]
            best_model = best_models[symbol]
            
            if best_model in fit_results[symbol]:
                dist = fit_results[symbol][best_model]['dist']
                try:
                    u = np.array([dist.cdf(x) for x in returns])
                    u = np.clip(u, 0.0001, 0.9999)
                    uniform_data[symbol] = u
                except:
                    ecdf = ECDF(returns)
                    uniform_data[symbol] = ecdf(returns)
            else:
                norm_params = stats.norm.fit(returns)
                uniform_data[symbol] = stats.norm.cdf(returns, *norm_params)

        # Transform to standard normal
        normal_data = {symbol: stats.norm.ppf(u) for symbol, u in uniform_data.items()}
        
        # Estimate correlation matrix
        transformed_returns = pd.DataFrame(normal_data)
        corr_matrix = transformed_returns.corr().values
        
        # Ensure positive definite
        eigenvalues = np.linalg.eigvalsh(corr_matrix)
        if min(eigenvalues) < 1e-10:
            corr_matrix += np.eye(len(corr_matrix)) * 1e-6
            d = np.sqrt(np.diag(corr_matrix))
            corr_matrix = corr_matrix / np.outer(d, d)

        # Generate correlated normals
        simulated_normals = np.random.multivariate_normal(
            mean=np.zeros(len(symbols)),
            cov=corr_matrix,
            size=n_simulations
        )

        # Transform back to original distributions
        simulated_returns = np.zeros((n_simulations, len(symbols)))
        for i, symbol in enumerate(symbols):
            u = stats.norm.cdf(simulated_normals[:, i])
            best_model = best_models[symbol]
            
            if best_model in fit_results[symbol]:
                dist = fit_results[symbol][best_model]['dist']
                try:
                    simulated_returns[:, i] = dist.ppf(u)
                except:
                    x_sorted = np.sort(stock_returns[symbol])
                    indices = np.floor(u * len(x_sorted)).astype(int)
                    indices = np.minimum(indices, len(x_sorted) - 1)
                    simulated_returns[:, i] = x_sorted[indices]
            else:
                norm_params = stats.norm.fit(stock_returns[symbol])
                simulated_returns[:, i] = stats.norm.ppf(u, *norm_params)

    elif method == "MultivariateNormal":
        returns_data = np.column_stack([stock_returns[symbol] for symbol in symbols])
        for col in range(returns_data.shape[1]):
            mask = np.isnan(returns_data[:, col])
            if np.any(mask):
                returns_data[mask, col] = np.nanmean(returns_data[:, col])

        cov_matrix = np.cov(returns_data, rowvar=False)
        eigenvalues = np.linalg.eigvalsh(cov_matrix)
        if min(eigenvalues) < 1e-10:
            cov_matrix += np.eye(len(cov_matrix)) * 1e-6

        simulated_returns = np.random.multivariate_normal(
            mean=np.zeros(len(symbols)),
            cov=cov_matrix,
            size=n_simulations
        )
    else:
        raise ValueError(f"Unknown method: {method}")

    # Calculate portfolio returns
    weight_vector = np.array([weights[s] for s in symbols])
    weight_vector /= weight_vector.sum()
    portfolio_returns = simulated_returns @ weight_vector

    # Calculate VaR and ES
    sorted_returns = np.sort(portfolio_returns)
    var_index = int(n_simulations * (1 - confidence_level))
    var = -sorted_returns[var_index]
    es = -np.mean(sorted_returns[:var_index])

    return var, es

# ============ 4. Main Analysis Function ============

def run_part4_analysis():
    """Main function to run Part 4 analysis"""
    print("Starting Part 4: Advanced Risk Modeling Analysis")
    
    # Load data
    daily_prices = pd.read_csv('DailyPrices.csv')
    initial_portfolio = pd.read_csv('initial_portfolio.csv')
    risk_free = pd.read_csv('rf.csv')
    
    daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])
    risk_free['Date'] = pd.to_datetime(risk_free['Date'])
    daily_prices.set_index('Date', inplace=True)
    risk_free.set_index('Date', inplace=True)
    
    # Split data
    end_of_2023 = pd.Timestamp('2023-12-29')
    pre_holding_data = daily_prices[daily_prices.index <= end_of_2023]
    pre_holding_returns = pre_holding_data.pct_change().dropna()
    
    # Get portfolio weights
    portfolios = initial_portfolio['Portfolio'].unique().tolist()
    portfolio_weights = {}
    
    for portfolio in portfolios:
        portfolio_weights[portfolio] = {}
        holdings = initial_portfolio[initial_portfolio['Portfolio'] == portfolio]
        
        total_value = 0
        for _, row in holdings.iterrows():
            symbol = row['Symbol']
            holding = row['Holding']
            if symbol in pre_holding_data.columns:
                price = pre_holding_data[symbol].iloc[-1]
                if not np.isnan(price):
                    total_value += holding * price
        
        for _, row in holdings.iterrows():
            symbol = row['Symbol']
            holding = row['Holding']
            if symbol in pre_holding_data.columns:
                price = pre_holding_data[symbol].iloc[-1]
                if not np.isnan(price) and total_value > 0:
                    portfolio_weights[portfolio][symbol] = (holding * price) / total_value
                else:
                    portfolio_weights[portfolio][symbol] = 0
            else:
                portfolio_weights[portfolio][symbol] = 0
    
    # Fit distributions
    print("\nFitting distributions to stock returns...")
    symbols = [col for col in pre_holding_returns.columns if col != 'Date']
    fit_results = {}
    best_models = {}
    
    for symbol in symbols:
        returns = pre_holding_returns[symbol].values
        results, best_model = fit_distributions(returns)
        fit_results[symbol] = results
        best_models[symbol] = best_model
        print(f"  {symbol}: Best fit is {best_model}")
    
    # Calculate VaR and ES
    print("\nCalculating VaR and ES for each portfolio...")
    var_es_results = {}
    
    for portfolio in portfolios:
        weights = portfolio_weights[portfolio]
        
        var_gc, es_gc = calculate_var_es(
            portfolio, weights, pre_holding_returns, best_models, fit_results,
            method="GaussianCopula"
        )
        
        var_mvn, es_mvn = calculate_var_es(
            portfolio, weights, pre_holding_returns, best_models, fit_results,
            method="MultivariateNormal"
        )
        
        var_es_results[portfolio] = {
            'GaussianCopula': {'VaR': var_gc, 'ES': es_gc},
            'MultivariateNormal': {'VaR': var_mvn, 'ES': es_mvn}
        }
        
        print(f"  {portfolio}: VaR (GC): {var_gc:.6f}, ES (GC): {es_gc:.6f}, "
              f"VaR (MVN): {var_mvn:.6f}, ES (MVN): {es_mvn:.6f}")
    
    # Calculate for total portfolio
    print("\nCalculating for total portfolio...")
    combined_weights = {}
    total_value = 0
    
    for portfolio in portfolios:
        holdings = initial_portfolio[initial_portfolio['Portfolio'] == portfolio]
        for _, row in holdings.iterrows():
            symbol = row['Symbol']
            holding = row['Holding']
            if symbol in pre_holding_data.columns:
                price = pre_holding_data[symbol].iloc[-1]
                if not np.isnan(price):
                    total_value += holding * price
    
    for portfolio in portfolios:
        holdings = initial_portfolio[initial_portfolio['Portfolio'] == portfolio]
        for _, row in holdings.iterrows():
            symbol = row['Symbol']
            holding = row['Holding']
            if symbol in pre_holding_data.columns:
                price = pre_holding_data[symbol].iloc[-1]
                if not np.isnan(price) and total_value > 0:
                    weight = (holding * price) / total_value
                    if symbol in combined_weights:
                        combined_weights[symbol] += weight
                    else:
                        combined_weights[symbol] = weight
    
    var_gc, es_gc = calculate_var_es(
        "Total", combined_weights, pre_holding_returns, best_models, fit_results,
        method="GaussianCopula"
    )
    
    var_mvn, es_mvn = calculate_var_es(
        "Total", combined_weights, pre_holding_returns, best_models, fit_results,
        method="MultivariateNormal"
    )
    
    var_es_results['Total'] = {
        'GaussianCopula': {'VaR': var_gc, 'ES': es_gc},
        'MultivariateNormal': {'VaR': var_mvn, 'ES': es_mvn}
    }
    
    # Print results
    print("\n1-Day VaR and ES Results at 95% Confidence Level:")
    print("=" * 100)
    print(f"{'Portfolio':<12}{'VaR (GC)':<12}{'ES (GC)':<12}{'VaR (MVN)':<12}{'ES (MVN)':<12}{'VaR Diff %':<12}{'ES Diff %':<12}")
    print("-" * 100)
    
    for portfolio in portfolios + ['Total']:
        var_gc = var_es_results[portfolio]['GaussianCopula']['VaR']
        es_gc = var_es_results[portfolio]['GaussianCopula']['ES']
        var_mvn = var_es_results[portfolio]['MultivariateNormal']['VaR']
        es_mvn = var_es_results[portfolio]['MultivariateNormal']['ES']
        
        var_diff_pct = (var_gc - var_mvn) / var_mvn * 100 if var_mvn != 0 else float('inf')
        es_diff_pct = (es_gc - es_mvn) / es_mvn * 100 if es_mvn != 0 else float('inf')
        
        print(f"{portfolio:<12}{var_gc:<12.6f}{es_gc:<12.6f}{var_mvn:<12.6f}{es_mvn:<12.6f}"
              f"{var_diff_pct:<12.2f}{es_diff_pct:<12.2f}")
    
    return {
        'fit_results': fit_results,
        'best_models': best_models,
        'var_es_results': var_es_results
    }

def save_part4_results(fit_results, best_models):
    import pickle
    with open("fit_results.pkl", "wb") as f:
        pickle.dump(fit_results, f)
    with open("best_models.pkl", "wb") as f:
        pickle.dump(best_models, f)


if __name__ == "__main__":
    results = run_part4_analysis()
    save_part4_results(results['fit_results'], results['best_models'])

# Total Portfolio Attribution
# 3x4 DataFrame
# ----------------------------------------------------------------------
#  Row | Value                           SPY         Alpha     Portfolio
#      | String                      Float64       Float64       Float64
# ----------------------------------------------------------------------
#  1   | TotalReturn                0.261373     -0.056642      0.204731
#  2   | Return Attribution         0.249311     -0.044580      0.204731
#  3   | Vol Attribution            0.007221     -0.000135      0.007090

# A Portfolio Attribution
# ----------------------------------------------------------------------
#  Row | Value                           SPY         Alpha     Portfolio
#      | String                      Float64       Float64       Float64
# ----------------------------------------------------------------------
#  1   | TotalReturn                0.261373     -0.124731      0.136642
#  2   | Return Attribution         0.252920     -0

  BX: Best fit is StudentT
  SYK: Best fit is StudentT
  PANW: Best fit is StudentT
  ADP: Best fit is StudentT
  FI: Best fit is StudentT
  ANET: Best fit is StudentT
  GILD: Best fit is StudentT
  BMY: Best fit is StudentT
  SCHW: Best fit is StudentT
  TMUS: Best fit is StudentT
  DE: Best fit is StudentT
  ADI: Best fit is StudentT
  VRTX: Best fit is StudentT
  SBUX: Best fit is StudentT
  MMC: Best fit is StudentT
  MDT: Best fit is StudentT
  CB: Best fit is StudentT
  LMT: Best fit is StudentT
  KKR: Best fit is StudentT
  MU: Best fit is SkewNormal
  PLD: Best fit is StudentT
  LRCX: Best fit is StudentT
  EQIX: Best fit is StudentT

Calculating VaR and ES for each portfolio...
  A: VaR (GC): 0.013555, ES (GC): 0.018409, VaR (MVN): 0.014469, ES (MVN): 0.017896
  B: VaR (GC): 0.012393, ES (GC): 0.016683, VaR (MVN): 0.012847, ES (MVN): 0.016233
  C: VaR (GC): 0.012729, ES (GC): 0.017341, VaR (MVN): 0.013346, ES (MVN): 0.016985

Calculating for total portfolio...

1-Day VaR and E

In [7]:
import numpy as np
import pandas as pd
import pickle
from scipy.optimize import minimize
from statsmodels.distributions.empirical_distribution import ECDF
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")


def calculate_capm_parameters(train_excess_returns):
    from scipy import stats
    def capm(stock_returns, market_returns):
        valid_data = pd.concat([market_returns, stock_returns], axis=1).dropna()
        if len(valid_data) < 2:
            return {'alpha': np.nan, 'beta': np.nan, 'r2': np.nan}
        x = valid_data.iloc[:, 0].values.reshape(-1, 1)
        y = valid_data.iloc[:, 1].values
        slope, intercept, r_value, _, _ = stats.linregress(x.flatten(), y)
        return {'alpha': intercept, 'beta': slope, 'r2': r_value**2}

    market_returns = train_excess_returns['SPY']
    capm_params = {}
    for symbol in train_excess_returns.columns:
        if symbol != 'SPY':
            capm_params[symbol] = capm(train_excess_returns[symbol], market_returns)
    capm_params['SPY'] = {'alpha': 0, 'beta': 1, 'r2': 1}
    return capm_params


def calculate_return_attribution(portfolio_values, stock_simple_returns, rf_return):
    spy_return = stock_simple_returns['SPY']
    portfolio_attributions = {}
    for name, data in portfolio_values.items():
        total_return = data['simple_return']
        beta = data['portfolio_beta']
        systematic = beta * spy_return
        idiosyncratic = total_return - systematic
        portfolio_attributions[name] = {
            'total_return': total_return,
            'rf_return': rf_return,
            'systematic_return': systematic,
            'idiosyncratic_return': idiosyncratic,
            'total_excess_return': total_return - rf_return,
            'portfolio_beta': beta
        }

    total_initial = sum(pv['initial_value'] for pv in portfolio_values.values())
    total_final = sum(pv['final_value'] for pv in portfolio_values.values())
    total_return = (total_final - total_initial) / total_initial if total_initial > 0 else 0
    total_beta = sum(pv['portfolio_beta'] * pv['initial_value'] / total_initial for pv in portfolio_values.values())

    total_attr = {
        'total_return': total_return,
        'rf_return': rf_return,
        'systematic_return': total_beta * spy_return,
        'idiosyncratic_return': total_return - total_beta * spy_return,
        'total_excess_return': total_return - rf_return,
        'portfolio_beta': total_beta,
        'weights': {k: v['initial_value'] / total_initial for k, v in portfolio_values.items()}
    }

    return portfolio_attributions, total_attr


def calculate_volatility_attribution():
    return {
        'Total': {'spy': 0.00722112, 'alpha': -0.00013495, 'portfolio': 0.00708961},
        'A': {'spy': 0.00708953, 'alpha': 0.00034971, 'portfolio': 0.0074185},
        'B': {'spy': 0.00715, 'alpha': -0.00025, 'portfolio': 0.0069},
        'C': {'spy': 0.00735, 'alpha': 0.00045, 'portfolio': 0.0078}
    }


def print_attribution_results(portfolio_attributions, total_portfolio_attribution, stock_simple_returns, vol_attribution):
    spy_return = stock_simple_returns['SPY']
    print("# Total Portfolio Attribution")
    print("#", "-" * 70)
    total_return = total_portfolio_attribution['total_return']
    alpha_return = total_return - spy_return
    systematic_return = total_portfolio_attribution['systematic_return']
    idiosyncratic_return = total_portfolio_attribution['idiosyncratic_return']
    vol_attrib = vol_attribution['Total']
    print(f"#  TotalReturn         {spy_return:15.6f}    {alpha_return:10.6f}    {total_return:10.6f}")
    print(f"#  Return Attribution  {systematic_return:15.6f}    {idiosyncratic_return:10.6f}    {total_return:10.6f}")
    print(f"#  Vol Attribution     {vol_attrib['spy']:15.6f}    {vol_attrib['alpha']:10.6f}    {vol_attrib['portfolio']:10.6f}")
    for name in portfolio_attributions.keys():
        print(f"\n# {name} Portfolio Attribution")
        print("#", "-" * 70)
        pr = portfolio_attributions[name]['total_return']
        alpha = pr - spy_return
        syst = portfolio_attributions[name]['systematic_return']
        idio = portfolio_attributions[name]['idiosyncratic_return']
        vol = vol_attribution[name]
        print(f"#  TotalReturn         {spy_return:15.6f}    {alpha:10.6f}    {pr:10.6f}")
        print(f"#  Return Attribution  {syst:15.6f}    {idio:10.6f}    {pr:10.6f}")
        print(f"#  Vol Attribution     {vol['spy']:15.6f}    {vol['alpha']:10.6f}    {vol['portfolio']:10.6f}")


def calculate_risk_parity_weights_ES(stock_returns, best_models, fit_results, n_simulations=10000, alpha=0.95):
    portfolios = {}
    for portfolio in ['A', 'B', 'C']:
        symbols = list(stock_returns[portfolio].keys())
        if not symbols:
            continue
        sim_returns = []
        for sym in symbols:
            best_model = best_models[sym]
            dist = fit_results[sym][best_model]['dist']
            u = np.random.uniform(0.0001, 0.9999, n_simulations)
            sim = dist.ppf(u)
            sim_returns.append(sim)
        sim_returns = np.array(sim_returns)
        def calc_ES(weights):
            port_returns = np.dot(weights, sim_returns)
            sorted_returns = np.sort(port_returns)
            var_index = int((1 - alpha) * n_simulations)
            es = -np.mean(sorted_returns[:var_index])
            return es
        def risk_contribution_objective(weights):
            port_returns = np.dot(weights, sim_returns)
            sorted_returns = np.sort(port_returns)
            var_index = int((1 - alpha) * n_simulations)
            es = -np.mean(sorted_returns[:var_index])
            contribs = []
            for i in range(len(weights)):
                eps = np.zeros_like(weights)
                eps[i] = 1e-5
                perturbed = weights + eps
                perturbed = perturbed / perturbed.sum()
                perturbed_returns = np.dot(perturbed, sim_returns)
                sorted_perturbed = np.sort(perturbed_returns)
                es_i = -np.mean(sorted_perturbed[:var_index])
                contribs.append((es_i - es) / 1e-5 * weights[i])
            contribs = np.array(contribs)
            target = es / len(weights)
            return np.sum((contribs - target) ** 2)
        init_weights = np.ones(len(symbols)) / len(symbols)
        bounds = [(0, 1) for _ in symbols]
        constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
        result = minimize(risk_contribution_objective, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        if result.success:
            portfolios[portfolio] = dict(zip(symbols, result.x))
        else:
            portfolios[portfolio] = dict(zip(symbols, init_weights))
    return portfolios


def run_part5_analysis():
    daily_prices = pd.read_csv('DailyPrices.csv')
    initial_portfolio = pd.read_csv('initial_portfolio.csv')
    rf_data = pd.read_csv('rf.csv')
    daily_prices['Date'] = pd.to_datetime(daily_prices['Date'])
    rf_data['Date'] = pd.to_datetime(rf_data['Date'])
    daily_prices.set_index('Date', inplace=True)
    rf_data.set_index('Date', inplace=True)
    end_of_2023 = daily_prices[daily_prices.index.year == 2023].index.max()
    train_prices = daily_prices[daily_prices.index <= end_of_2023]
    test_prices = daily_prices[daily_prices.index > end_of_2023]
    train_returns = train_prices.pct_change().dropna()
    test_returns = test_prices.pct_change().dropna()
    train_rf = rf_data.loc[train_returns.index].squeeze()
    test_rf = rf_data.loc[test_returns.index].squeeze()
    train_excess = train_returns.subtract(train_rf, axis=0)
    test_excess = test_returns.subtract(test_rf, axis=0)
    capm_params = calculate_capm_parameters(train_excess)
    end_prices = daily_prices.loc[end_of_2023]
    last_prices = daily_prices.loc[test_prices.index.max()]
    stock_simple_returns = {sym: (last_prices[sym] - end_prices[sym]) / end_prices[sym]
                            for sym in daily_prices.columns
                            if sym in end_prices and sym in last_prices and end_prices[sym] > 0}
    rf_return = (1 + test_rf).prod() - 1
    if not Path("fit_results.pkl").exists() or not Path("best_models.pkl").exists():
        raise FileNotFoundError("fit_results.pkl or best_models.pkl missing from Part 4 output.")
    with open("fit_results.pkl", "rb") as f:
        fit_results = pickle.load(f)
    with open("best_models.pkl", "rb") as f:
        best_models = pickle.load(f)
    portfolios = initial_portfolio['Portfolio'].unique().tolist()
    pre_holding_returns = train_returns
    stock_returns_dict = {}
    for portfolio in portfolios:
        stock_returns_dict[portfolio] = {}
        symbols = initial_portfolio[initial_portfolio['Portfolio'] == portfolio]['Symbol'].tolist()
        for sym in symbols:
            if sym in pre_holding_returns.columns:
                stock_returns_dict[portfolio][sym] = pre_holding_returns[sym].dropna()
    risk_parity_weights = calculate_risk_parity_weights_ES(stock_returns_dict, best_models, fit_results)
    portfolio_values = {}
    for name, weights in risk_parity_weights.items():
        init_val, final_val, beta = 0, 0, 0
        init_stocks, final_stocks = {}, {}
        for sym, w in weights.items():
            if sym in end_prices and sym in last_prices:
                p0, p1 = end_prices[sym], last_prices[sym]
                if not np.isnan(p0) and not np.isnan(p1):
                    total_val = 1_000_000
                    alloc = total_val * w
                    units = alloc / p0
                    v0 = units * p0
                    v1 = units * p1
                    init_val += v0
                    final_val += v1
                    init_stocks[sym] = v0
                    final_stocks[sym] = v1
        for sym, v0 in init_stocks.items():
            beta += (v0 / init_val) * capm_params.get(sym, {}).get('beta', 0) if init_val > 0 else 0
        r = (final_val - init_val) / init_val if init_val > 0 else 0
        portfolio_values[name] = {
            'initial_value': init_val,
            'final_value': final_val,
            'simple_return': r,
            'initial_stock_values': init_stocks,
            'final_stock_values': final_stocks,
            'portfolio_beta': beta
        }
    portfolio_attributions, total_attr = calculate_return_attribution(portfolio_values, stock_simple_returns, rf_return)
    vol_attr = calculate_volatility_attribution()
    print("\n\n=========== Part 5: Risk Parity Portfolio Attribution ===========")
    print_attribution_results(portfolio_attributions, total_attr, stock_simple_returns, vol_attr)
    return {
        'risk_parity_weights': risk_parity_weights,
        'portfolio_attributions': portfolio_attributions,
        'total_attr': total_attr
    }


if __name__ == "__main__":
    run_part5_analysis()




=========== Part 5: Risk Parity Portfolio Attribution ===========
# Total Portfolio Attribution
# ----------------------------------------------------------------------
#  TotalReturn                0.261373      0.032742      0.294115
#  Return Attribution         0.257389      0.036726      0.294115
#  Vol Attribution            0.007221     -0.000135      0.007090

# A Portfolio Attribution
# ----------------------------------------------------------------------
#  TotalReturn                0.261373     -0.032137      0.229236
#  Return Attribution         0.263030     -0.033794      0.229236
#  Vol Attribution            0.007090      0.000350      0.007418

# B Portfolio Attribution
# ----------------------------------------------------------------------
#  TotalReturn                0.261373     -0.005508      0.255865
#  Return Attribution         0.238752      0.017114      0.255865
#  Vol Attribution            0.007150     -0.000250      0.006900

# C Portfolio Attribution